In [1]:
# load model from 2.2 (last epoch)
from keras.models import load_model
import os
model_path = r'C:\Users\Carla Pugh\PycharmProjects\VideoSlicing\jupyter notebooks\model2_2_chkp'
filename = 'weights.100-0.39.hdf5'
fullpath = os.path.join(model_path,  filename)
model = load_model(fullpath)

In [2]:
# load test data
from keras_video import VideoFrameGenerator
import os
import glob
import keras

glob_pattern='../data_v3_model_2.2/test_data_temp/{classname}/*.avi'

classes = ['suture_throws', 'hand_ties', 'thread_cuts']
classes.sort()

# Create video frame generator
# some global params
SIZE = (100, 100) # height and width of frame pxl by pxl
CHANNELS = 3 # RGB or whatever
NBFRAME = 5 # num frames in sequence 
BS = 8 # Batch size

train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val = 0.000001,
    split_test=0.999999,
    shuffle=False,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    use_frame_cache=True)

class hand_ties, validation count: 0, test count: 49, train count: 1
class suture_throws, validation count: 0, test count: 49, train count: 1
class thread_cuts, validation count: 0, test count: 49, train count: 1
Total data: 3 classes for 3 files for train


In [3]:
test = train.get_test_generator()

Total data: 3 classes for 147 files for test


In [4]:
files=test.files

In [5]:
a = model.predict_generator(test)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [6]:
len(a)

144

In [7]:
a

array([[9.99980569e-01, 5.95429265e-06, 1.34583825e-05],
       [9.99349535e-01, 4.15001909e-04, 2.35368323e-04],
       [9.99017239e-01, 6.38679543e-04, 3.44061700e-04],
       [9.99576032e-01, 9.25617496e-05, 3.31356918e-04],
       [9.99645829e-01, 9.64385617e-05, 2.57769861e-04],
       [9.99890327e-01, 2.04396529e-05, 8.92857643e-05],
       [3.35783958e-02, 9.13156390e-01, 5.32652065e-02],
       [9.99957919e-01, 1.59160190e-05, 2.60514971e-05],
       [9.99691725e-01, 1.22321348e-04, 1.86036006e-04],
       [9.99679565e-01, 1.58381386e-04, 1.62107608e-04],
       [9.99602258e-01, 1.97264701e-04, 2.00592200e-04],
       [9.98891413e-01, 3.88541579e-04, 7.20036216e-04],
       [9.99430120e-01, 3.18741222e-04, 2.51180958e-04],
       [9.99680758e-01, 1.57959381e-04, 1.61313117e-04],
       [9.97145832e-01, 3.19099723e-04, 2.53516948e-03],
       [9.99672771e-01, 1.39346899e-04, 1.87942322e-04],
       [9.99691606e-01, 1.22346173e-04, 1.86072532e-04],
       [9.99980569e-01, 5.94886

In [8]:
classes

['hand_ties', 'suture_throws', 'thread_cuts']

In [9]:
classes = ['hand_ties', 'suture_throws', 'thread_cuts']
def predict_maneuver(probs):
    mx = -1
    mx_index = -1
    for i in range(len(probs)):
        if probs[i] > mx:
            mx = probs[i]
            mx_index = i
    return classes[mx_index]

In [10]:
# naive way: I am going to assume that the last 5 files are not predicted

def get_name(fpath):
    fname = fpath.split('\\')[1][:-4]
    pre = fname.split('_')[0]
    for name in classes:
        if pre in name:
            return name
        
predictions = []
y_true = []
y_pred = []
for i in range(len(a)):
    predictions.append(
    {
        'correct_label': get_name(files[i]),
        #'correct_label': files[i],
        'prediction': predict_maneuver(a[i])
    })    
    y_true.append(get_name(files[i]))
    y_pred.append(predict_maneuver(a[i]))

In [11]:
predictions

[{'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'suture_throws'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'prediction': 'hand_ties'},
 {'correct_label': 'hand_ties', 'pre

In [12]:
from sklearn.metrics import accuracy_score

In [17]:
print('accuracy: {}'.format(accuracy_score(y_true, y_pred)))

accuracy: 0.9305555555555556


In [14]:
thred_pred_cnt = 0
actual_thred_cnt = 0
for i in range(len(predictions)):
    if predictions[i]['correct_label'] == 'thread_cuts':
        actual_thred_cnt += 1
    if predictions[i]['prediction'] == 'thread_cuts':
        thred_pred_cnt += 1
print('fraction of predictions that are thread cuts: {}'.format(thred_pred_cnt/len(predictions)))
print('fraction of inputs that are thread cuts: {}'.format(actual_thred_cnt/len(predictions)))

fraction of predictions that are thread cuts: 0.2569444444444444
fraction of inputs that are thread cuts: 0.3194444444444444
